Estudiaremos en el presente Notebook las cotizaciones y las distintas carácterísticas que componen los ETFs de BlackRock, conocidos como iShares.
Todos ellos serán los cotizados en **USD-$**, para poder realizar comparativas y agrupaciones de forma más sencilla.

Debido a esta condición, añadida a un mínimo de 20 años desde su salida al mercado para tener un número considerable de registros, nos encontramos ante **87 ETFs** distintos para analizar.


# Data Project

## Dataset

Nos quedará un Dataset compuesto por el histórico de la capitalización de cada ETF de iShare, que añadiremos a nuestro entorno de desarrollo en la carpeta de *pre-processed*, donde incluiremos todos los csv.

Estos csv han sido generados despúes de un primer paso de darle calidad a nuestros datos a partir de un script de Python que transforma los datos de la carpeta *raw* a los csv de *pre-processed*, preparándolos para su uso en KDB+.

En nuestro mismo directorio, recibiremos otro csv, donde estará la información a cargar para la tabla Resume, también generado por Python.


  > Tabla **Historical**: Registra el histórico de cotización de los ETFs. 
  >  - ***date***: La fecha del registro del histórico.    
  > - ***nav_value***: El valor de la participación en esa fecha.    
  > - ***shares***: El nº de participaciones en el mercado del ETF en esa fecha.  
  > - ***ticker***: El ticker identificativo del ETF.

  > Tabla **Resume**: Se trata de la información general del los ETFs de iShare. 
  >  - ***ticker***: El identificativo de cada ETF de iShares.    
  > - ***inception_date***: La fecha a la que sale al mercado el ETF.    
  > - ***net_assets***: El total de capitalización del ETF en el momento de sacar los datos.  
  > - ***equity_class***: Muestra el tipo de activo que compone el ETF. *Large/Mid Cap - All Cap - Large Cap - Mid Cap - Small Cap*
  > - ***region***: Muestra la región de dónde son las empresas. *Asia Pacific - North America  - Europe - Latin America - Global - Middle East and Africa*
  > - ***market***: Muestra el tipo de mercado que tiene las empresas. *Developed - Emerging*

1. Crearemos en un primer momento las tablas vacías.
2. Importaremos desde los csv la información y la insertaremos en nuestras tablas.

In [ ]:
historical:([]date:`date$();nav_value:`float$();shares:`long$();ticker:`g#`$()) // ... ticker:`g#`$()
resume_etf:([ticker:`$()]inception_date:`date$();net_assets:`float$();
            equity_class:`$();region:`$();market:`$())
/@[`historical;`ticker;`g#];
meta historical

## Data Processing - Filter and Quality
Nuestros datos están disponibles en la carpeta *Data*, pero se presentan en un formato de Excel y con información poco valiosa y muy dispersa para el estudio. Por tanto consideramos que se encuentran en un proceso raw en nuestro proyecto de Ingeniería de Datos.

Por ello contamos con 2 scripts a realizar, donde filtraremos, renombraremos y daremos calidad a los datos, para posteriormente poder estudiarlos de mejor manera.

Pero antes de ello realizaremos la instalación de las librerías correspondientes, actualizadas en el *requirements.txt*:

In [ ]:
system "pip install -r requirements.txt"

In [ ]:
system "python3 Data/script_etf_a_csv.py Data/RawExcels/ Data/iShares-UnitedStates.xlsm"
system "python3 Data/script_resume_etf.py Data/iShares-UnitedStates.xlsm"

## Data Ingest

Procedemos a importar los archivos de los ETFs que se encuentran en la ruta : *Data/PreProcessed/etf/* y los cargaremos  para poder operar con ellos.

In [ ]:
csv_files: system "ls Data/PreProcessed/etf/*.csv";

Insertamos en nuestra tabla **'historical'**  usando un iterador y las características de programación funcional y su compilación de derecha a izquierda que nos aporta el lenguaje.

Importamos los datos del csv, actualizamos la columna ticker con el nombre que queremos y realizamos un upsert sobre la tabla.

##### Ingesta en Historical

In [ ]:
{ ticker_name: ssr[ssr[; "Data/PreProcessed/etf/"; ""] x; ".csv"; ""]; 
    `historical upsert update ticker:`$ticker_name from ("DFJ"; enlist csv) 0: `$x;
 } each csv_files;

##### Ingesta en Resume_etf

In [ ]:
`resume_etf insert ("SDFSSS"; enlist csv) 0: `$"Data/PreProcessed/resume.csv";

# New Data and Metadata

Procederemos a crear columnas en nuestras tablas con intención de recopilar información sobre el histórico y poder analizar posteriormente con mayor información y datos.

## Daily Returns:
 Añadiremos la diferencia de precios diaria en la tabla de *Historical* para poder calcular la volatilidad total.

 Por ello, lo primero que haremos será ordenar por ticker y fecha paraque se calcule de manera correcta los daily returns.

In [ ]:
historical: `ticker`date xasc historical;
historical: update daily_retns: (nav_value-prev nav_value)%prev nav_value by ticker from historical

## Daily Performance

Guardaremos la variación de precio diaria de los ticker.

In [ ]:
historical: `ticker`date xasc historical;
historical: update daily_performance: nav_value-prev nav_value by ticker from historical

## Volatility
Añadimos la volatilidad total por ticker a la tabla *Historical*.

Tomaremos la fórmula de la Volatilidad cómo: 
> Volatility = σ<sub>period</sub> * √T<sub>period</sub>

En mercados financieros los años se tratan de 252 días de mercados abiertos.

Para ello usaremos la función creada a continuación, que recibe 2 parámetros, del registro del que se va a procear y el periodo de la volatilidad del que se va a calcular.

El cálculo se hace manera diaria, cogiendo los periodos previos a cada día de una manera dinámica. 


In [ ]:
calc_vol: {
    ticker1: x`ticker;           
    initial_date: x`date;        
    other_date: "d"$(initial_date - 365 * y);      
    limit_date: first (select inception_date from resume_etf where ticker = ticker1)[`inception_date];    
    a: $[other_date <= limit_date;
            0n;
            [
                filtered: select daily_retns from historical where ticker = ticker1, 
                            date within (other_date; initial_date);
                dev filtered[`daily_retns]* sqrt[252 * y]
            ]
        ]  
 }

### Historical Volatility - 5 Years

In [ ]:
historical: update daily_5yvol: {calc_vol[x;5]} each historical from historical;

### Historical Volatility - 3 Years

In [ ]:
historical: update daily_3yvol: {calc_vol[x;3]} each historical from historical;

### Historical Volatility - 1 Year

In [ ]:
historical: update daily_1yvol: {calc_vol[x;1]} each historical from historical;

### Historical Volatility - 6 Months

In [ ]:
historical: update daily_6mvol: {calc_vol[x;0.5]} each historical from historical;

### Historical Volatility - 3 Months

In [ ]:
historical: update daily_3mvol: {calc_vol[x;0.25]} each historical from historical;

Finalmente revisamos que se haya añadido correctamente a *historical*.

## Quartiles of Volatility

A la hora de estimar y agrupar nos interesará saber en que puntos nuestras volatilidades está en cada cuartil. Por ello añadiremos a la tabla resume_etf los valores de los quartiles de 3 y 5 años para la hora de calcular los % tener las fronteras de referencia.

In [ ]:
etfs: exec ticker from resume_etf;
{
    volts: exec daily_5yvol from historical where ticker=x;
    filtP: volts where not null volts;
    sortedVolts: asc filtP;
    iP: count sortedVolts;
    q1IndexP: "i"$floor (iP + 1) % 4;
    q2IndexP: "i"$floor (iP + 1) % 2;
    q3IndexP: "i"$floor 3 * (iP + 1) % 4;
    q1P: $[q1IndexP > 0; sortedVolts[q1IndexP - 1]; null];
    q2P: $[q2IndexP > 0; sortedVolts[q2IndexP - 1]; null];
    q3P: $[q3IndexP > 0; sortedVolts[q3IndexP - 1]; null];
    resume_etf: update Q1_5yV: q1P, Q2_5yV: q2P, Q3_5yV: q3P from `resume_etf where ticker=x;
 } each etfs;

{
    volts: exec daily_3yvol from historical where ticker=x;
    filtP: volts where not null volts;
    sortedVolts: asc filtP;
    iP: count sortedVolts;
    q1IndexP: "i"$floor (iP + 1) % 4;
    q2IndexP: "i"$floor (iP + 1) % 2;
    q3IndexP: "i"$floor 3 * (iP + 1) % 4;
    q1P: $[q1IndexP > 0; sortedVolts[q1IndexP - 1]; null];
    q2P: $[q2IndexP > 0; sortedVolts[q2IndexP - 1]; null];
    q3P: $[q3IndexP > 0; sortedVolts[q3IndexP - 1]; null];
    resume_etf: update Q1_3yV: q1P, Q2_3yV: q2P, Q3_3yV: q3P from `resume_etf where ticker=x;
 }each etfs;

{
    volts: exec daily_1yvol from historical where ticker=x;
    filtP: volts where not null volts;
    sortedVolts: asc filtP;
    iP: count sortedVolts;
    q1IndexP: "i"$floor (iP + 1) % 4;
    q2IndexP: "i"$floor (iP + 1) % 2;
    q3IndexP: "i"$floor 3 * (iP + 1) % 4;
    q1P: $[q1IndexP > 0; sortedVolts[q1IndexP - 1]; null];
    q2P: $[q2IndexP > 0; sortedVolts[q2IndexP - 1]; null];
    q3P: $[q3IndexP > 0; sortedVolts[q3IndexP - 1]; null];
    resume_etf: update Q1_1yV: q1P, Q2_1yV: q2P, Q3_1yV: q3P from `resume_etf where ticker=x;
 }each etfs;

{
    volts: exec daily_6mvol from historical where ticker=x;
    filtP: volts where not null volts;
    sortedVolts: asc filtP;
    iP: count sortedVolts;
    q1IndexP: "i"$floor (iP + 1) % 4;
    q2IndexP: "i"$floor (iP + 1) % 2;
    q3IndexP: "i"$floor 3 * (iP + 1) % 4;
    q1P: $[q1IndexP > 0; sortedVolts[q1IndexP - 1]; null];
    q2P: $[q2IndexP > 0; sortedVolts[q2IndexP - 1]; null];
    q3P: $[q3IndexP > 0; sortedVolts[q3IndexP - 1]; null];
    resume_etf: update Q1_6mV: q1P, Q2_6mV: q2P, Q3_6mV: q3P from `resume_etf where ticker=x;
 }each etfs;

{
    volts: exec daily_3mvol from historical where ticker=x;
    filtP: volts where not null volts;
    sortedVolts: asc filtP;
    iP: count sortedVolts;
    q1IndexP: "i"$floor (iP + 1) % 4;
    q2IndexP: "i"$floor (iP + 1) % 2;
    q3IndexP: "i"$floor 3 * (iP + 1) % 4;
    q1P: $[q1IndexP > 0; sortedVolts[q1IndexP - 1]; null];
    q2P: $[q2IndexP > 0; sortedVolts[q2IndexP - 1]; null];
    q3P: $[q3IndexP > 0; sortedVolts[q3IndexP - 1]; null];
    resume_etf: update Q1_3mV: q1P, Q2_3mV: q2P, Q3_3mV: q3P from `resume_etf where ticker=x;
 }each etfs;

In [ ]:
meta historical
meta resume_etf

## Moving Averages

Calcularemos las medias móviles de ciertos periodos:
 - 200 días
 - 200 meses


### SMA 200 Days

Usamos la ordenación ascendente para coger los últimos 200 registros desde el principio de los registros disponibles.

In [ ]:
historical: `ticker`date xasc historical;
historical: update d200sma:mavg[200;nav_value] by ticker from historical

### SMA 200 Months

Usamos la ordenación ascendente + last para coger el ultimo día del mes empezando por los primeros registros disponibles.

In [ ]:
temp: select last date, last nav_value by ticker, month:`month$date from historical
temp: update m200sma: 200 mavg nav_value by ticker from temp
temp: delete month from temp
historical: historical lj `ticker`date xkey temp

In [ ]:
meta historical

## Period Performance
Calcularemos los rendimientos a **FUTURO** de cada momento del etf en los siguientes periodos, en un valor porcentual(%):
- 5 años
- 3 años
- 1 año
- 6 meses
- 3 meses

Para ello usaremos la siguiente función, que realiza un cálculo dinámico de los registros.

In [ ]:
calc_performance: {
    ticker1: x`ticker;           
    initial_date: x`date;        
    other_date: "d"$(initial_date + 365 * y);
    limit_date: exec max date from historical where ticker = ticker1;
    a: $[other_date > limit_date;
            0n;
            [
                ant: first (select nav_value from historical 
                        where date = initial_date, ticker = ticker1)`nav_value;
                act: first (select nav_value from historical 
                        where date = other_date, ticker = ticker1)`nav_value;
                (act-ant)%ant
            ]
        ]  
 }

### Performance - 5 Years

In [ ]:
historical: update performance_5y: {calc_performance[x;5]} each historical from historical;

### Performance - 3 Years

In [ ]:
historical: update performance_3y: {calc_performance[x;3]} each historical from historical;

### Performance - 1 Year

In [ ]:
historical: update performance_1y: {calc_performance[x;1]} each historical from historical;

### Performance - 6 Months

In [ ]:
historical: update performance_6m: {calc_performance[x;0.5]} each historical from historical;

### Performance - 3 Months

In [ ]:
historical: update performance_3m: {calc_performance[x;0.25]} each historical from historical;

Al final añadimos a la table *historical* los datos calculados por cada valor de cotización de los etf.

In [ ]:
meta historical

## Drawdowns

Calcularemos:
- 5 años
- 3 años
- 1 año

In [ ]:
each_drawdown: {
    ticker1: x`ticker;
    initial_date: x`date;
    other_date: "d"$(initial_date - 365 * y);
    limit_date: first (select inception_date from resume_etf where ticker = ticker1)[`inception_date];
    
    a: $[other_date <= limit_date;
        0n;
        [
            filtered: select date, nav_value from historical 
                        where ticker = ticker1, date within (other_date; initial_date);
            max_value: max filtered`nav_value;
            min_value: first (exec min nav_value from filtered 
                                    where date >= first date where nav_value = max_value);
            (max_value - min_value) % max_value
        ]
    ]
 }

### Drawdown - 5 Years

In [ ]:
historical: update drawdown_5y: {each_drawdown[x;5]} each historical from historical;

### Drawdown - 3 Years

In [ ]:
historical: update drawdown_3y: {each_drawdown[x;3]} each historical from historical;

### Drawdown - 1 Year

In [ ]:
historical: update drawdown_1y: {each_drawdown[x;1]} each historical from historical;

Finalmente añadimos a la table *resume_etf* los datos calculados por cada etf de los 3 drawdowns.

In [ ]:
meta historical

## Inflows/Outflows

Calculamos las variaciones de valor total de los etf, con la ampliación/salida de dinero en el mercado del etf por la variación mensual de las participaciones en circulación.

In [ ]:
historical: `ticker`date xasc historical
temp: select last date, last nav_value, last shares by ticker, m:`month$date from historical
temp: update in_out_flow_per: 100 * (shares - prev shares) % prev shares,
    in_out_flow_val: nav_value * (shares - prev shares)    
    by ticker from temp;
temp: delete m from temp
historical: historical lj `ticker`date`shares xkey temp

# Save to Data Warehouse

Una vez hemos generado toda la información y las tablas están listas, estas las guardaremos en disco.

Esto nos dará esa modularidad y persistencia de los datos para otros procesos futuros, pudiendo acudir a unos  datos ya elaborados y normalizados sin tener que acudir a los datos en bruto del DataLake. 

Guardaremos nuestras tablas en la carpeta de tablas del Data Warehouse.

### Guardado como Binarios

In [ ]:
\ts `:Data/DataWarehouse/Tables/historical set historical
\ts `:Data/DataWarehouse/Tables/resume_etf set resume_etf

In [ ]:
meta resume_etf

### Guardado Particionado

In [ ]:
\ Tenemos que particionar y crear un enum diferenciado para todos los simbolos
\ts `:Data/DataWarehouse/Tables/historicalSplayed/historical/ set 
    .Q.en[`:Data/DataWarehouse/Tables/historicalSplayed/] historical;

In [ ]:
\ts `:Data/DataWarehouse/Tables/resume_etfSplayed/resume_etf set 
    .Q.enxs[`:Data/DataWarehouse/Tables/resume_etfSplayed/] resume_etf;

# Data Analysis: "The Ideal Conditions for ETFs to invest in!"

La idea es analizar los datos recopilados y poder acotar bajo que *parámetros/características* nuestros etf se comportan de cierto modo. 

Intentaremos agrupar y de una manera estimar:

> - ¿Cómo es la volatilidad o los drawdowns cuando llego a pérdidas superiores al 25%?
> - ¿Y por otro lado, cómo es esa volatilidad o drawdown cuando llego a beneficios superiores al 25%?

El objetivo: **Poder hacer prediciones con los datos a día de hoy que nos permita a partir de poder agrupar y estimar esos valores con la intención de MINIMIZAR EL RIESGO con compras/ventas.**

Nos encontramos ahora con unas tablas enriquecidas con mucha información de la siguiente manera:

> Tabla **Historical**: A parte de las ya existentes **date**, **nav_value**, **shares**, **ticker** se añade:
> - ***Daily Returns***: El aumento/pérdida de valor de manera % de un ETF respecto al dato previo.
> - ***Volatility: 5y,3y,1y,6m,3m***: La volatilidad de los precios del ETF en esos periodos previos de cada registro histórico.
> - ***Performance: 5y,3y,1y,6m,3m***: La variación del valor del ETF en esos periodos **próximos** de cada registro histórico.
> - ***Drawdowns: 5y,3y,1y***; La caída máxima del precio de cotización en esos periodos previos de cada registro histórico.
> - ***SMA 200 Days***: El cálculo de la medía móvil de los últimos 200 días por cada ETF.
> - ***SMA 200 Months***: El cálculo de la medía móvil de los últimos 200 meses(último dato del mes disponible) por cada ETF.
> - ***Inflows/Outflows %***: Las entradas/salidas de dinero a nuestro ETF de manera mensual, con la ampliación/reducción de participaciones(shares) disponibles en el mercado.
> - ***Inflows/Outflows Value***: El valor en $ de la variación de participaciones en el mercado mensual.

> Tabla **Resume**: A parte de las ya existentes **ticker**, **inception_date**, **net_assets**, **equity_class**, **region**, **market** se añade:
> - ***Maximum/Minimum:*** Las fechas de cada ETF donde se alcanza esos valores.

Para ello usaremos el [visual](/dataAnalytics.ipynb), donde estudiaremos estas cuestiones en profundidad y generaremos visuales.